# DATA20001 Deep Learning - Group Project
## Image project

**Due Thursday, May 20, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imread` from `matplotlib.pyplot`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

## Get the data

In [194]:
import sys
import os
from os.path import join
from os.path import abspath
from os.path import split

import torch
import torch.nn as nn
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import zipfile

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

root_dir = os.getcwd()
if root_dir not in sys.path:
    sys.path.append(root_dir)
    
train_path = 'train'

train_path_images = os.path.join(train_path, 'images')
train_path_annotations = os.path.join(train_path, 'annotations')

# check if we need to unzip data
try:
    os.listdir(train_path_images)
except OSError as e:
    data_folder_name = 'image-training-corpus+annotations'
    DATA_FOLDER_DIR = os.path.abspath(os.path.join(root_dir, data_folder_name))

    data_zip_name = 'dl2018-image-proj.zip'
    DATA_ZIP_DIR = os.path.abspath(os.path.join(DATA_FOLDER_DIR, data_zip_name))

    with zipfile.ZipFile(DATA_ZIP_DIR) as zip_f:
        zip_f.extractall(train_path)
        


class CustomDataset(Dataset):
    def __init__(self, train_path_images, train_path_annotations):
        # store path to images to get images dynamically
        self.train_path_images = train_path_images
        self.length = len(os.listdir(train_path_images))
        self.n_categories = len(os.listdir(train_path_annotations))+1
        self.categories = {} #{'bird': 0, 'baby': 1, ...}
        
        # get annotations
        one_hot_encoding_other = np.zeros(self.n_categories)
        one_hot_encoding_other[-1] = 1
        self.labels_one_hot_encoding = {}
        self.labels_one_hot_encoding['other'] = one_hot_encoding_other
        self.categories['other'] = self.n_categories
        
        self.annotations = {}
        
        for file_index, annotation_file in enumerate(os.listdir(train_path_annotations)):
            category, _ = annotation_file.split('.')
            self.categories[category] = file_index
            
            self.labels_one_hot_encoding[category] = np.zeros(self.n_categories)
            self.labels_one_hot_encoding[category][file_index] = 1
            with open(os.path.join(train_path_annotations, annotation_file)) as f:
                for line in f:
                    img_id = int(line.replace('\n', ''))
                    if img_id in self.annotations:
                        # set additional class in label
                        self.annotations[img_id][file_index] = 1
                    else: 
                        # initial label with one hot encoding
                        self.annotations[img_id] = self.labels_one_hot_encoding[category]
                      
        # fill 'other' for unlabeled images
        for i in range(1, self.length+1):
            if i not in self.annotations:
                self.annotations[i] = one_hot_encoding_other
        
        # class weights based on counts
        self.weights = [0 for i in range(self.n_categories)]
        for annotation in self.annotations.values():
            for i, e in enumerate(annotation):
                if e:
                    self.weights[i] += 1
        # scaling
        self.weights = torch.Tensor(list(map(lambda x: x / (custom_dataset.weights.sum()-x), custom_dataset.weights)))      
        

        # Transformations
        #self.center_crop = transforms.CenterCrop(100)

        self.grayscale = transforms.Grayscale(num_output_channels=1)
        self.normalize = transforms.Normalize(0.5, 0.5)
        self.to_tensor = transforms.ToTensor()
        
    def __getitem__(self, index):
        index += 1 # image labels start with 1
        label = self.annotations[index]
        img = Image.open(os.path.join(self.train_path_images, 'im{}.jpg'.format(index)))
        
        # When you call the transform for the second time it calls __call__() and applies the transform 
        #data = self.center_crop(data)
        
        # img is PIL Image
        img = self.grayscale(img)
        
        # img to tensor
        img = self.to_tensor(img)
        img = self.normalize(img)

        #label = self.to_tensor(label)
        return img, label

    def __len__(self):
        return len(self.annotations)

custom_dataset = CustomDataset(train_path_images, train_path_annotations)


In [196]:
custom_dataset.weights.sum()

tensor(1.1051)

The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Training

In [226]:
if torch.cuda.is_available():
    print('Using GPU!')
    device = torch.device('cuda')
else:
    print('Using CPU')
    device = torch.device('cpu')
    
num_epochs = 5
num_classes = len(os.listdir(train_path_annotations))+1
batch_size = 1000
learning_rate = 0.001

######## layer 1 ########
config_layer1_conv = {
    'in_channels': 1,
    'out_channels': 16,
    'kernel_size': 15,   # first layer kernel might be a bit bigger
    'stride': 1,
    'padding': 0,   
}
config_layer1_pool = {
    'kernel_size': 5,
    'stride': 2          # to get size down
}

######## layer 2 ########
config_layer2_conv = {
    'in_channels': 16,
    'out_channels': 32,
    'kernel_size': 5,
    'stride': 1,
    'padding': 0,   
}
config_layer2_pool = {
    'kernel_size': 5,
    'stride': 2
}

######## layer 3 ########
config_layer3_conv = {
    'in_channels': 32,
    'out_channels': 16,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0,   
}
config_layer3_pool = {
    'kernel_size': 5,
    'stride': 2
}

# Define data loader
dataset_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                                batch_size=batch_size,
                                                shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(**config_layer1_conv),
            nn.ReLU(),
            nn.MaxPool2d(**config_layer1_pool)
        )
            
        self.layer2 = nn.Sequential(
            nn.Conv2d(**config_layer2_conv),
            nn.ReLU(),
            nn.MaxPool2d(**config_layer2_pool)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(**config_layer3_conv),
            nn.ReLU(),
            nn.MaxPool2d(**config_layer3_pool)
        )
            
        self.drop_out = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(9 * 9 * 16, 248) # 9 * 9 * 16 = 1296, 248
        self.fc2 = nn.Linear(248, num_classes)
        self.fc3 = nn.Sigmoid()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out
    
    
model = CNN()
# Loss and optimizer
#criterion = nn.BCEWithLogitsLoss(pos_weight=custom_dataset.weights)
cirterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(dataset_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i_batch, (images, labels) in enumerate(dataset_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy  
        # TODO: accurace per image not per class, but right now acc per image looks too devastating
        correct = float(((torch.sigmoid(outputs.data).data > 0.5) == labels).sum() / num_classes)

        acc_list.append(correct / batch_size)
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i_batch + 1, total_step, loss.item(),
                          (correct / batch_size) * 100))


Using GPU!
0.308
Epoch [1/5], Step [1/20], Loss: 0.6897, Accuracy: 30.80%
0.304
Epoch [1/5], Step [2/20], Loss: 0.6710, Accuracy: 30.40%
0.287
Epoch [1/5], Step [3/20], Loss: 0.6383, Accuracy: 28.70%
0.291
Epoch [1/5], Step [4/20], Loss: 0.5690, Accuracy: 29.10%
0.293
Epoch [1/5], Step [5/20], Loss: 0.5223, Accuracy: 29.30%
0.295
Epoch [1/5], Step [6/20], Loss: 0.5100, Accuracy: 29.50%
0.402
Epoch [1/5], Step [7/20], Loss: 0.5125, Accuracy: 40.20%
0.608
Epoch [1/5], Step [8/20], Loss: 0.5139, Accuracy: 60.80%
0.696
Epoch [1/5], Step [9/20], Loss: 0.5189, Accuracy: 69.60%
0.705
Epoch [1/5], Step [10/20], Loss: 0.5124, Accuracy: 70.50%
0.71
Epoch [1/5], Step [11/20], Loss: 0.5136, Accuracy: 71.00%
0.717
Epoch [1/5], Step [12/20], Loss: 0.5161, Accuracy: 71.70%
0.722
Epoch [1/5], Step [13/20], Loss: 0.5191, Accuracy: 72.20%
0.711
Epoch [1/5], Step [14/20], Loss: 0.5126, Accuracy: 71.10%
0.72
Epoch [1/5], Step [15/20], Loss: 0.5180, Accuracy: 72.00%
0.728
Epoch [1/5], Step [16/20], Loss: 0

KeyboardInterrupt: 

In [228]:
outputs.data

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.2838e-33, 1.6314e-36, 8.9739e-37,  ..., 7.8437e-33, 2.0810e-31,
         1.2857e-29],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [227]:
((torch.sigmoid(outputs.data).data > 0.5) == labels).sum()

tensor(10370)

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".  [See more here on how to load the model back](https://github.com/pytorch/pytorch/blob/761d6799beb3afa03657a71776412a2171ee7533/docs/source/notes/serialization.rst) if you want to continue training later.

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')